In [1]:
from utils.utils import *

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


## Epidemics

### GKAN

In [7]:
model_path = './saved_models_optuna/model-epidemics-gkan/epidemics-new-data/2/gkan'

In [8]:
pysr_model = lambda : get_pysr_model(
    model_selection="score",
    random_state = 0,
    deterministic=True,
    parallelism='serial',
    n_iterations=150
)

symb_model_black_box = fit_black_box_from_kan(
    model_path=model_path,
    n_g_hidden_layers=2,
    n_h_hidden_layers=2,
    theta=-np.inf,
    pysr_model=pysr_model,
    sample_size=9000,
    message_passing=False
)

/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


In [9]:
symb_model_black_box

\sum_{j}( 0.500805746365327*x_j*(0.9996049 - x_i)) - 0.49989313*x_i

### MLP-based baseline

In [11]:
model_path_mpnn = './saved_models_optuna/model-epidemics-mpnn/epidemics-new-data/1'

In [12]:
pysr_model = lambda : get_pysr_model(
    model_selection="score",
    random_state = 0,
    deterministic=True,
    parallelism='serial',
    n_iterations=150
)

mpnn_symb = fit_mpnn(
    model_path=model_path_mpnn,
    pysr_model=pysr_model,
    sample_size=9000,
    message_passing=False
)


/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


In [13]:
mpnn_symb

\sum_{j}( x_i*(0.49942818 - 0.49865368*x_j)) - 0.5019439*x_i